In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from delta.tables import DeltaTable

# Slowly Changing Dimension - Initial and Incremental

In [0]:
df = spark.read.format("csv")\
        .option("header",True)\
        .option("inferSchema",True)\
        .load("/FileStore/RealTime/rawcsv")

In [0]:
df.limit(5).display()

p_id,p_name,p_category,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11
1,cookies,food,null,null,null,null,null,null,null,null,null
2,almonds,food,null,null,null,null,null,null,null,null,null
3,Toothpase,merchandise,null,null,null,null,null,null,null,null,null
4,earphones,electronics,null,null,null,null,null,null,null,null,null
5,oil,merchandise,null,null,null,null,null,null,null,null,null


In [0]:
df=df.select('p_id','p_name','p_category').filter(col('p_id').isNotNull())

In [0]:
df.display()

p_id,p_name,p_category
1,cookies,food
2,almonds,food
3,Toothpase,merchandise
4,earphones,electronics
5,oil,merchandise
6,shirt,merchandise


In [0]:
dbutils.widgets.text("initial_run", 'second_time')

In [0]:
if(dbutils.widgets.get("initial_run")=='second_time'):
    delta_table = DeltaTable.forPath(spark,"/FileStore/RealTime/rawcsvsink")
    
    delta_table.alias("trg").merge(df.alias("src"), "trg.p_id = src.p_id")\
                            .whenMatchedUpdateAll()\
                            .whenNotMatchedInsertAll()\
                            .execute()
    
else:
    df.write.format("delta")\
        .mode("append")\
        .save("dbfs:/FileStore/RealTime/rawcsvsink")

In [0]:
df = spark.read.format("delta").load("dbfs:/FileStore/RealTime/rawcsvsink")
df.display()

p_id,p_name,p_category
1,cookies,food
2,almonds,food
3,Toothpase,merchandise
4,earphones,electronics
5,oil,merchandise
6,shirt,merchandise


In [0]:
#dbutils.fs.rm("dbfs:/FileStore/RealTime/rawcsvsink", recurse=True)